In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd
from fastai.text import *
import gc

In [4]:
ls_class = []
ls_class_defn = []
ls_q = []
with open('5_label.txt','r',encoding="utf-8") as fin:
    for line in fin.readlines():
        ww = line.replace('\n','').split(' ')
        tt = ww[0].split(':')
        ls_class.append(tt[0])
        ls_class_defn.append(tt[1])
        ls_q.append(" ".join(ww[1:]))
        
df = pd.DataFrame({'LABEL': ls_class, 'LABEL_DEFINITION': ls_class_defn, 'QUESTION': ls_q})

In [5]:
df.head()

,LABEL,LABEL_DEFINITION,QUESTION
0,DESC,manner,How did serfdom develop in and then leave Russ...
1,ENTY,cremat,What films featured the character Popeye Doyle ?
2,DESC,manner,How can I find a list of celebrities ' real na...
3,ENTY,animal,What fowl grabs the spotlight after the Chines...
4,ABBR,exp,What is the full form of .com ?


In [6]:
df_train = df.drop(columns=['LABEL_DEFINITION'])

In [7]:
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=2019)

In [8]:
train_df.loc[:,'is_valid'] = [False]*train_df.shape[0]
val_df.loc[:,'is_valid'] = [True]*val_df.shape[0]

/home/sankarshan/miniconda3/envs/dl/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/sankarshan/miniconda3/envs/dl/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
del df_train, df
gc.collect()

7

In [10]:
df_train = pd.concat([train_df, val_df])

In [11]:
df_train.head()

,LABEL,QUESTION,is_valid
5173,DESC,How can you become an FBI agent ?,False
3801,DESC,What is capitalism according to Max Weber ?,False
1475,LOC,Where can I find all the information I need to...,False
2298,LOC,Where can I find book reviews of `` Turbulent ...,False
2208,NUM,How many children does Ray Davies of the Kinks...,False


In [12]:
?TextList.split_from_df

Signature:
TextList.split_from_df(
    ['self', 'col:Union[int, Collection[int], str, Collection[str]]=2'],
)
Docstring: Split the data from the `col` in the dataframe in `self.xtra`.
File:      ~/miniconda3/envs/dl/lib/python3.6/site-packages/fastai/data_block.py
Type:      function


In [22]:
data_class = (TextList.from_df(df_train, cols='QUESTION')
              .split_from_df(col='is_valid')
              .label_from_df(cols='LABEL')
              .databunch())

In [23]:
data_class.show_batch()

text,target
"xxbos xxmaj who was named xxmaj xxunk of the xxmaj ocean xxmaj seas and xxmaj xxunk and xxmaj governor xxmaj general of all the islands he xxunk discover , and also xxunk xxunk ? ? of all xxunk of his xxunk .",HUM
xxbos xxmaj what xxmaj french designer declared : `` xxmaj the jean is the xxunk ! xxmaj it is a dictator ! xxmaj it is xxunk xxunk . xxmaj the jean must be stopped ! '' ?,HUM
"xxbos xxmaj what is the current population for these countries : xxmaj france , xxmaj spain , xxmaj italy , xxmaj xxunk , xxmaj xxunk , xxmaj germany , xxmaj switzerland , the xxmaj xxunk ?",NUM
"xxbos xxmaj how much does it cost , average or xxunk is xxunk , to produce and send xxunk mail xxunk in the xxup us ? , xxup dt xxup cd xxup nn xxup nn ,",NUM
"xxbos xxmaj how many people on the xxunk were killed from the xxunk of xxmaj pan xxmaj am xxmaj flight xxunk over xxmaj xxunk , xxmaj xxunk , xxmaj december 21 , xxup cd .",NUM


In [28]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 40 # max number of words in a question to use

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class CNN_Text(nn.Module):
    
    def __init__(self, args):
        super(CNN_Text, self).__init__()
        self.args = args
        
        V = args.embed_num
        D = args.embed_dim
        C = args.class_num
        Ci = 1
        Co = args.kernel_num
        Ks = args.kernel_sizes

        self.embed = nn.Embedding(V, D)
        # self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        '''
        self.conv13 = nn.Conv2d(Ci, Co, (3, D))
        self.conv14 = nn.Conv2d(Ci, Co, (4, D))
        self.conv15 = nn.Conv2d(Ci, Co, (5, D))
        '''
        self.dropout = nn.Dropout(args.dropout)
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)  # (N, Co, W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        
        if self.args.static:
            x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        '''
        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)
        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)
        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)
        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)
        '''
        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        return logit